In [1]:
%cd ../

/u02/thanh/workplace/plasma


In [2]:
import time

import plasma.parallel_processing.communicators as ppc
import plasma.parallel_processing.queues as ppq
from plasma.functional import partials, AutoPipe, LambdaPipe

/u02/thanh/workplace/plasma/.env/RnD/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def compute(x):
    time.sleep(0.5)
    return f'compute: {x}'

def compute2(x):
    time.sleep(0.5)
    return f'compute2: {x + 'c'}'

class ComputeClass(AutoPipe):

    def __init__(self, hihi, haha):
        super().__init__()
    
    def run(self, tmp):
        return 'ComputeClass: ' + tmp + 'd'

In [4]:
out_queue= ppq.TransferQueue().register_callback(print)

class Flow(ppc.AsyncFlow):

    def __init__(self):
        super().__init__()

        distributor = ppc.distributors.IteratorDistributor()
        compute3 = ComputeClass(1, 2)
        self.chain(
            (ppq.ThreadQueue(), distributor),
            (distributor, compute, ppq.ProcessQueue(5)),
                (compute, print, ppq.TransferQueue()),
                (compute, out_queue),
                (compute, compute2, ppq.ProcessQueue(5)),
                    (compute2, print),
                    (compute2, out_queue),
                    (compute2, compute3, ppq.ProcessQueue()),
                        (compute3, print),
                        (compute3, out_queue)
        )

flow = Flow()
flow

[ThreadQueue(name=None, runner=1, id=129864570583568)]

  |->(IteratorDistributor, id=129864570582896)

    |->[ProcessQueue(name=None, runner=5, id=129864570583904)]

      |->(<function compute at 0x761c73874f40>, id=129864569409344)

        |->[TransferQueue(name=None, runner=1, id=129864569186384)]

          |->(<built-in function print>, id=129870803330096)

        |->[TransferQueue(name=None, runner=1, id=129870752728800)]

        |->[ProcessQueue(name=None, runner=5, id=129864569187984)]

          |->(<function compute2 at 0x761c73874b80>, id=129864569408384)

            |->[TransferQueue(name=None, runner=1, id=129864569186384)]

              |->(<built-in function print>, id=129870803330096)...

            |->[TransferQueue(name=None, runner=1, id=129870752728800)]

            |->[ProcessQueue(name=None, runner=1, id=129864569188304)]

              |->(ComputeClass, id=129864570583232)

                |->[TransferQueue(name=None, runner=1, id=129864569186384)]

    

In [5]:
with flow:
   for i in range(10):
      flow.put([i] * 2)

compute: 0
compute: 0
compute: 1
compute: 2
compute: 1
compute: 2
compute: 3
compute2: compute: 0c
compute2: compute: 0c
compute: 4
compute2: compute: 2c
compute2: compute: 1c
compute: 3
compute: 4
compute2: compute: 1c
ComputeClass: compute2: compute: 0cd
ComputeClass: compute2: compute: 0cd
ComputeClass: compute2: compute: 2cd
ComputeClass: compute2: compute: 1cd
ComputeClass: compute2: compute: 1cd
compute: 5
compute: 5
compute2: compute: 2c
ComputeClass: compute2: compute: 2cd
compute2: compute: 3c
compute: 6
ComputeClass: compute2: compute: 3cd
ComputeClass: compute2: compute: 4cd
compute2: compute: 4c
compute: 6
compute: 7
ComputeClass: compute2: compute: 3cd
ComputeClass: compute2: compute: 4cd
compute2: compute: 4c
compute2: compute: 3c
compute: 7
compute: 8
compute2: compute: 5c
ComputeClass: compute2: compute: 5cd
compute2: compute: 6c
ComputeClass: compute2: compute: 5cd
compute: 8
compute2: compute: 5c
ComputeClass: compute2: compute: 6cd
compute2: compute: 7c
compute: 9
co

In [6]:
flow.alive()

False

In [7]:
out_queue.run()

TransferQueue(
  name=None,
  num_runner=1,
)

compute: 0
compute: 0
compute: 1
compute: 2
compute: 1
compute: 2
compute: 3
compute2: compute: 0c
compute2: compute: 0c
compute2: compute: 2c
compute: 4
compute2: compute: 1c
compute2: compute: 1c
compute: 3
compute: 4
ComputeClass: compute2: compute: 0cd
ComputeClass: compute2: compute: 0cd
ComputeClass: compute2: compute: 2cd
ComputeClass: compute2: compute: 1cd
ComputeClass: compute2: compute: 1cd
compute: 5
compute: 5
compute2: compute: 2c
ComputeClass: compute2: compute: 2cd
compute2: compute: 3c
ComputeClass: compute2: compute: 3cd
compute: 6
compute2: compute: 4c
ComputeClass: compute2: compute: 4cd
compute: 6
compute: 7
compute2: compute: 3c
ComputeClass: compute2: compute: 3cd
ComputeClass: compute2: compute: 4cd
compute2: compute: 4c
compute: 7
compute: 8
compute2: compute: 5c
ComputeClass: compute2: compute: 5cd
compute2: compute: 6c
ComputeClass: compute2: compute: 5cd
ComputeClass: compute2: compute: 6cd
compute2: compute: 5c
compute: 9
compute: 8
compute2: compute: 7c
co

In [8]:
out_queue.release()